In [40]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

## Loading and preparing label maps

In [4]:
with open('../../temp/label_matching_coco_oi_ws_300.pkl', 'rb') as fin:
    labelmap = pickle.load(fin)

In [5]:
idx2mid_coco = labelmap['idx2mid_coco']
idx2mid_oi = labelmap['idx2mid_oi']
idx2mid_ws = labelmap['idx2mid_ws']
mid2idx_coco = labelmap['mid2idx_coco']
mid2idx_oi = labelmap['mid2idx_oi']
mid2idx_ws = labelmap['mid2idx_ws']
mid2name_coco = labelmap['mid2name_coco']
mid2name_oi = labelmap['mid2name_oi']
mid2name_ws = labelmap['mid2name_ws']
name2mid_coco = labelmap['name2mid_coco']
name2mid_oi = labelmap['name2mid_oi']
name2mid_ws = labelmap['name2mid_ws']
common_mids_coco = labelmap['common_mids_coco']
common_mids_ws = labelmap['common_mids_ws']


In [6]:
coco_mids = set(mid2idx_coco.keys())
oi_mids = set(mid2idx_oi.keys())
ws_mids = set(mid2idx_ws.keys())


In [7]:
with open('../../temp/all_labels_coco_oi_ws_300.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [8]:
with open('../../results/det_results_postproc_12.pkl', 'rb') as fin:
    det_results_sorted = pickle.load(fin)

## Visualizing Results

In [9]:
mid2name_extended = {}
extended_classes = set()
for key in coco_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_coco[key] + f' ({abv})'
for key in oi_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_oi[key] + f' ({abv})'
for key in ws_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_ws[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [10]:
det_results_forviz = copy.deepcopy(det_results_sorted)

In [11]:
for key in det_results_forviz:
    for i in range(len(det_results_forviz[key]['detection_classes'])):
        label = det_results_forviz[key]['detection_classes'][i] + '/' + det_results_forviz[key]['detection_models'][i]
        det_results_forviz[key]['detection_classes'][i] = mid2idx_extended[label]


In [12]:
print(det_results_forviz[key]['detection_classes'])

[3470, 2459, 2142, 3470, 1840, 4082, 2075, 1730, 2088, 2052, 4230, 3973, 109, 2627, 454, 2043, 86, 2209, 4247, 3813, 3673, 1003, 3984, 292, 547, 4135, 2985, 4193, 2197, 3779, 144, 1676, 3375, 173, 3586, 4055, 562, 1898, 4231, 213, 3412, 1152, 4079, 4445, 4130, 4428, 1327, 3618, 2775, 3412, 1155, 1876, 963, 3178, 3541, 3178, 195, 1316, 985, 1274, 3835, 2245, 929, 3258, 2459, 1274, 861, 3258, 2348]


In [13]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [19]:
with open('../../../../data/seedling/seedling_keyframes/keyframes.txt', 'r') as fin:
    test_img_subpaths = [line.strip() for line in fin]

In [20]:
test_img_paths = ['../../../../data/seedling/seedling_keyframes/keyframes/' + item for item in test_img_subpaths]

In [34]:
filename_to_filepath = {item.split('/')[-1]: item for item in test_img_paths}

In [14]:
root_docs = ['IC0015MEL','IC0015N8B','IC0015R7B','IC0015S3J','IC0015S43','IC0015S4O','IC0015S7Y','IC0015S83','IC0015SWP','IC0015LGD']

In [16]:
leaf_docs = []
for item in root_docs:
    leaf_docs += root_to_leaf[item]

In [17]:
vid_ids = [item.split('.')[0] for item in leaf_docs if 'mp4' in item]

In [37]:
vid_to_shot = {}
docid_to_filepath = {}
with open('../../../../data/seedling/seedling_keyframes/msb.txt', 'r') as fin:
    for line in fin:
        row = line.split()
        shotname = row[1]
        vidname = row[1].split('_')[0]
        
        if vid_to_shot.get(vidname) == None:
            vid_to_shot[vidname] = []
        vid_to_shot[vidname].append(shotname)
        
        docid = row[1]
        filename = row[0] + '_' + docid.split('_')[-1] + '.png'
        filepath = filename_to_filepath[filename]
        docid_to_filepath[docid] = filepath
        

In [38]:
docid_to_filepath[vid_to_shot[vid_ids[0]][0]]

'../../../../data/seedling/seedling_keyframes/keyframes/cineast_metadata_part2/v_pGHkQQ8gHAoAp9qj/v_pGHkQQ8gHAoAp9qj_1.png'

In [39]:
def show(docid, thresh):
    imgid = docid + '.png'
    filename = docid_to_filepath[docid]  
    print(imgid)
    with open(filename, 'rb') as fin:
        #_ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      np.asarray(det_results_forviz[imgid]['detection_boxes_normalized']),
      det_results_forviz[imgid]['detection_classes'],
      det_results_forviz[imgid]['detection_scores'],
      category_index_extended,
      instance_masks=det_results_forviz[imgid].get('detection_masks'),
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()    

In [43]:
vidid = vid_ids[0]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video IC0019N0N ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [44]:
vidid = vid_ids[1]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video IC0019N3V ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [45]:
vidid = vid_ids[2]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video IC0019NB0 ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [46]:
vidid = vid_ids[3]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video IC0019NAH ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [47]:
vidid = vid_ids[4]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video HC0005KMS ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [48]:
vidid = vid_ids[5]
print(f'------ video {vidid} ------')
for docid in vid_to_shot[vidid]:
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        docid=fixed(docid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

------ video HC0005KS5 ------


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…